# Inteligencia ArtificialProyecto #2: Clasificación
## Inegrantes
    - Luis Berrospi
    - Pedro Dominguez
    - Carlos Guerrero

### Lectura de datos

### Reducción de dimensionalidad

### SVM

#### Estimación del error

#### Tabla de resumen

### KNN

#### Estimación del error

#### Tabla de resumen

### Decision tree

#### Estimación del error

#### Tabla de resumen